In [3]:
import pandas as pd

# Read the lookup table
data = pd.ExcelFile('lookupTable.xlsx')
lookupTable = pd.read_excel(data, 'previousVMMCSubmissions')
typesLookup = pd.read_excel(data,'types')

# Read the mcs dataset
dashboard = pd.ExcelFile("data.xlsx")

mcs = pd.read_excel(dashboard,"Statistics")

# Clean column names in lookupTable (remove 'field: ' part)
lookupTable.columns = [col.split(":")[1].strip() if ':' in col else col for col in lookupTable.columns]


# Identify common columns between lookupTable and mcs
common_columns = [col for col in mcs.columns if col in lookupTable.columns]

# Subset mcs to keep only the common columns
mcs_cleaned = mcs[common_columns]

# If lookupTable is empty, append rows from mcs_cleaned
if lookupTable.empty:
    lookupTable = mcs_cleaned.copy()
else:
    # If lookupTable is not empty, append the rows in mcs_cleaned that don't already exist
    lookupTable = pd.concat([lookupTable, mcs_cleaned]).drop_duplicates(subset=['facilityName', 'recordingMonth', 'year'], keep='last')

# Drop rows with NaN values
lookupTable = lookupTable[lookupTable['facilityName'].notna()]

lookupTable.columns = [
    'field: ' + col if col not in ['Delete(Y/N)', 'UID'] else col
    for col in lookupTable.columns
]

# Fill missing values with 0 for all columns except 'Delete(Y/N)' and 'field: UID'
columns_to_exclude = ['Delete(Y/N)', 'UID']

# Fill missing values with 0 in the columns that are not in the exclusion list
lookupTable = lookupTable.apply(lambda x: x.fillna(0) if x.name not in columns_to_exclude else x)

# Assign 'N' to every cell in the 'field: Delete(Y/N)' column
lookupTable['Delete(Y/N)'] = 'N'


# Now save the DataFrame to Excel
with pd.ExcelWriter('ZDIPlookup.xlsx', engine='xlsxwriter') as writer:
    # Save the sorted general data to the first sheet
    typesLookup.to_excel(writer, sheet_name='types', index=False)
    
    # Save the sorted AE data to a new sheet
    lookupTable.to_excel(writer, sheet_name='previousVMMCSubmissions', index=False)

In [4]:
lookupTable.to_excel('resultantLookupTable.xlsx')
lookupTable

,UID,Delete(Y/N),field: recordingMonth,field: year,field: facilityName,field: mc15-19,field: mc20-24,field: mc25-29,field: mc30-34,field: mc35-39,...,field: hivPositiveNC45-49,field: hivPositiveNC50,field: fu15-19,field: fu20-24,field: fu25-29,field: fu30-34,field: fu35-39,field: fu40-44,field: fu45-49,field: fu50
0,NaN,N,April,2025.0,Beula,0.0,6.0,5.0,0.0,0.0,...,0.0,0.0,0.0,6.0,5.0,0.0,0.0,0.0,0.0,0.0
1,NaN,N,April,2025.0,Natisa,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NaN,N,April,2025.0,Zengeza Clinic,85.0,75.0,69.0,10.0,0.0,...,0.0,0.0,85.0,75.0,69.0,10.0,0.0,0.0,0.0,0.0
